In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

#i grafici verranno visualizzati direttamente sotto le loro celle, inoltre è attivo l'autoreload del kernel

In [2]:
import sys
sys.path.append("C:\\Users\\celin\\AppData\\Local\\Programs\\Python\\Python312")
sys.path.append("/Users/francescoparino/Researchs/Mac2Mic/Util/")
sys.path.append("C:\\Users\\celin\\Desktop\\polito\\magistrale\\tesi\\flu_2023")
sys.path.append("./")
#queste istruzioni servono a dire a Python in quali percorsi bisogna cercare i moduli

In [3]:
#importiamo i moduli che ci servono
import pandas as pd #si usa per manipolare e analizzare dati: file excel, csv ecc.
import numpy as np #funzioni, array multidimensionali, calcoli complessi
from numpy import ones
import scipy as spy #si usa assieme a numpy: calcola integrali, minimizza e massimizza funzionali, trasformate fourier ecc
import os #permette di manipolare file
import copy
import pickle
import itertools
import datetime #fornisce classi e funzioni per manipolare date e orari
import matplotlib #si usa per creare grafici


from glob import glob #glob lo usiamo per dire a python che file ci serve, dando come input il percorso

import seaborn as sns #seaborn aggiunge opzioni grafiche a matplotlib
blues = np.array(sns.light_palette("blue"))[[-4,-1]]
purples = np.array(sns.light_palette("purple"))[[-4,-1]]

import matplotlib.patches as mpatches #oggetti geometrici (patches) per i grafici
from collections import Counter, defaultdict #fornisce delle classi utili a manipolare dati aggregati
from itertools import combinations

from tqdm import tqdm_notebook #taqaddum, fornisce barra di avanzamento per i cicli for

import collections, functools, operator 
from bidict import bidict
from sklearn.decomposition import NMF
from matplotlib.colors import LogNorm

from multiprocessing import Pool

In [4]:
#definisce una mappa di colori, ma non la usa mai? 

twoColor = np.array(sns.hls_palette(8, l=.4, s=.9))[[1,2]]
from matplotlib import colors

def array2cmpa(X):
    # Assuming array is Nx3, where x3 gives RGB values
    # Append 1's for the alpha channel, to make X Nx4

    X = np.c_[X,ones(len(X))]
    return colors.LinearSegmentedColormap.from_list('my_colormap', X)
twoColor = array2cmpa(twoColor)

coloMapMio = array2cmpa([(0.39471600258765605, 0.3011839071152027, 0.9088100597864414),
 (0.4776440582751795, 0.39692650086266157, 0.921303706524999),
 (0.5605721139627029, 0.4926690946101205, 0.9337973532635568),
 (0.6435001696502263, 0.5884116883575794, 0.9462910000021145),
 (0.7285014267299378, 0.6865478469487247, 0.9590969879091361),
 (0.8114294824174613, 0.7822904406961836, 0.9715906346476939),
 (0.8943575381049846, 0.8780330344436424, 0.9840842813862516),
 (0.95, 0.95, 0.95),
 (0.95, 0.95, 0.95),
 (0.8894900049789078, 0.9500428687418099, 0.8783502418622695),
 (0.7720924421399901, 0.8696585567920415, 0.754143441252265),
 (0.6546948793010723, 0.7892742448422732, 0.6299366406422604),
 (0.5343623773911816, 0.7068803250937606, 0.5026246700170056),
 (0.41696481455226386, 0.6264960131439923, 0.37841786940700106),
 (0.2995672517133462, 0.546111701194224, 0.2542110687969965),
 (0.18216968887442836, 0.4657273892444556, 0.13000426818699196)])

In [37]:
from engine import parameters, readFluDataset

In [8]:
fluProv = readFluDataset()

c:\Users\celin\Desktop\polito\magistrale\tesi\flu_2023\engine.py:76: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [9]:
fluProv

,anno,settimana,provincia,numero_infetti,data
57,2018,46,Torino,8098.60902,12-11-2018
64,2018,46,Vercelli,637.43936,12-11-2018
61,2018,46,Biella,656.54348,12-11-2018
63,2018,46,Verbano-Cusio-Ossola,577.88519,12-11-2018
62,2018,46,Novara,1318.68246,12-11-2018
...,...,...,...,...,...
79,2018,46,Siracusa,1091.70516,12-11-2018
74,2018,46,Oristano,83.47578,12-11-2018
46,2018,46,Monza e Brianza,1739.54106,12-11-2018
75,2018,46,Sud Sardegna,223.08471,12-11-2018


In [38]:
# tutorial sulle classi: una volta definita come sopra, bisogna crearne una istanza

# Creiamo un esempio di nh e Whk
nh = np.array([1000, 2000, 1500])
Whk = np.array([[0.1, 0.2, 0.1], [0.2, 0.3, 0.4], [0.1, 0.4, 0.2]])

# Creiamo un'istanza della classe parameters
params = parameters(nh, Whk)

In [39]:
params.eta

array([0.5, 0.3, 0.2])

In [50]:
# crea i compartimenti iniziali ricevendo come input una istanza della classe parameters e, volendo, anche gli esposti e infetti iniziali. 
# restituisce come output le matrici S, E, I, N, R all'istante t = 0
from datetime import datetime, timedelta

def initialStates(p, initialCaseI=None, initialCaseE=None, start_date=None):
    shape = (3, p.Whk.shape[0])  # Aggiungiamo una terza classe per gli anziani vaccinati
    
    I = np.zeros(shape)
    E = np.zeros(shape)
    N = np.zeros(shape)
    R = np.zeros(shape)
    
    if initialCaseI is not None:
        I[:2] = p.eta[:2].reshape(-1, 1).dot(initialCaseI.reshape(1, -1))
        
    if initialCaseE is not None:
        E[:2] = p.eta[:2].reshape(-1, 1).dot(initialCaseE.reshape(1, -1))
    
    Stot = np.ones(shape) * p.eta.reshape(-1,1).dot(p.nh.reshape(1,-1))
    S = Stot - E - I - N

    if start_date is not None:
        start_date = datetime.strptime(start_date, '%d-%m-%Y')
    else:
        start_date = datetime.now()
    
    return S, E, I, N, R, start_date

def oneStep(SEINR, p):
    S, E, I, N, R = SEINR
    Lambda, Nu, Beta, Gamma = p.Lambda, p.Nu, p.Beta, p.Gamma
    m, eta, ai, alpha, nh, b, Whk = p.m, p.eta, p.ai, p.alpha, p.nh, p.b, p.Whk
    
    aAvg = (eta * ai * alpha).sum()
    nhTilde = (1 - b * aAvg) * nh + (b * aAvg * (Whk * (nh.reshape(-1, 1))).sum(0))

    pht2 = (Whk * (((b * alpha * ai).reshape(-1, 1) * (I)).sum(0)).reshape(-1, 1)).sum(0)
    pht1 = ((1 - b * alpha * ai).reshape(-1, 1) * (I)).sum(0)
    ph = (1 / nhTilde) * (pht1 + pht2)
    
    qht2 = (Whk * (((b * alpha * ai).reshape(-1, 1) * (I)).sum(0)).reshape(-1, 1)).sum(0)
    qht1 = (((1 - b) * alpha * ai).reshape(-1, 1) * (I)).sum(0)
    qh = (1 / nhTilde) * (qht1 + qht2)
    
    PIt1 = (alpha * ai * (1 - b) * Lambda * m).reshape(-1, 1) * ph
    PIt2 = ((1 - (alpha * ai * b)) * Lambda * m).reshape(-1, 1) * qh
    PIt3 = Lambda * m * ((alpha * ai * b).reshape(-1, 1) * ((Whk * (ph + qh)).sum(1)))
    PI = PIt1 + PIt2 + PIt3
    
    St1 = (1 - PI) * S
    Et1 = (PI * S) + (1 - Nu) * E
    It1 = Nu * E + (1 - Beta) * I
    Nt1 = Beta * I + (1 - Gamma) * N
    Rt1 = R + Gamma * N 
    
    S = St1
    E = Et1
    I = It1
    N = Nt1    
    R = Rt1
    
    if S.min().min() < 0: 
        E += np.minimum(0, S)
        S = np.maximum(0, S)
            
    if E.min().min() < 0: 
        I += np.minimum(0, E)
        E = np.maximum(0, E)
    
    if I.min().min() < 0: 
        N += np.minimum(0, I)
        I = np.maximum(0, I)

    if N.min().min() < 0: 
        R += np.minimum(0, N)
        N = np.maximum(0, N)
        
    R = np.maximum(0, R)

    _nh = S.sum(0) + E.sum(0) + I.sum(0) + N.sum(0) + R.sum(0)
    assert np.isclose(nh, _nh, rtol=0, atol=1e-2).all(), f"population not preserved! {nh}, {_nh}"
    
    return S, E, I, N, R

# Funzione per calcolare i decessi
def calculate_deaths(R, mortality_rate):
    deaths = (R * mortality_rate.reshape(-1, 1)).sum(axis=0)
    return deaths

# Funzione per vaccinare gli anziani
def vaccinate_elders(S, p, vaccinate_rate):
    vaccinated_elderly_idx = 2  # Indice della nuova classe "anziani vaccinati"
    elderly_idx = 1  # Indice della classe "anziani"
    
    # Calcoliamo quanti anziani vaccinare
    num_elderly_to_vaccinate = S[elderly_idx] * vaccinate_rate
    
    # Spostiamo gli anziani vaccinati dal compartimento S degli anziani a quello degli anziani vaccinati
    S[vaccinated_elderly_idx] += num_elderly_to_vaccinate
    S[elderly_idx] -= num_elderly_to_vaccinate
    
    return S

# Funzione per salvare i risultati della simulazione
def save_results(filename, results):
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)

In [51]:
# Esempio di esecuzione della simulazione con tre classi di cittadini
nh = np.array([1000, 2000, 1500])
Whk = np.array([[0.3, 0.2, 0.5], [0.2, 0.4, 0.4], [0.5, 0.4, 0.1]])

params = parameters(nh, Whk)

initial_case_i = np.array([100, 50, 0])
initial_case_e = np.array([20, 10, 0])
start_date = '01-01-2020'

S, E, I, N, R, current_date = initialStates(params, initial_case_i, initial_case_e, start_date)

results = {
    'Day': [],
    'Province': [],
    'Suscettibili': [],
    'Esposti': [],
    'Infetti': [],
    'Non Contagiosi': [],
    'Rimossi': [],
    'Morti': []
}


# Vaccinazione degli anziani nel giorno 30
vaccinate_day = 30
vaccinate_rate = 0.5

for day in range(100):
    if day == vaccinate_day:
        S = vaccinate_elders(S, params, vaccinate_rate)
    
    S, E, I, N, R = oneStep((S, E, I, N, R), params)
    deaths = calculate_deaths(R, np.array([0.01, 0.05, 0.02]))  # Tasso di mortalità: [giovani, anziani, anziani vaccinati]
    
    for province in range(Whk.shape[0]):
        results['Day'].append(day)
        results['Province'].append(province)
        results['Suscettibili'].append(S[:, province].sum())
        results['Esposti'].append(E[:, province].sum())
        results['Infetti'].append(I[:, province].sum())
        results['Non Contagiosi'].append(N[:, province].sum())
        results['Rimossi'].append(R[:, province].sum())
        results['Morti'].append(deaths[province])

# Salvare i risultati in un file di testo
save_results('simulation_results.csv', results)

print("\nStati dopo un timestep:")
print("Suscettibili:\n", S)
print("Esposti:\n", E)
print("Infetti:\n", I)
print("Non Contagiosi:\n", N)
print("Rimossi:\n", R)


Stati dopo un timestep:
Suscettibili:
 [[0.04320812 0.120514   0.08923347]
 [0.38676295 1.06481248 0.79357039]
 [1.87564147 4.95251566 3.63115671]]
Esposti:
 [[0.17298489 0.7284282  0.59384158]
 [0.21177493 0.77477263 0.61786754]
 [0.63991904 2.32980918 1.82533032]]
Infetti:
 [[15.2268361  45.61324813 36.54496384]
 [ 9.25617174 24.19878445 19.13477463]
 [13.18505967 41.36676739 33.00162714]]
Non Contagiosi:
 [[277.50498294 588.85653978 447.66801535]
 [149.75804223 277.13879228 204.97154235]
 [135.76077787 321.08890822 248.9774545 ]]
Rimossi:
 [[207.05198795 364.68126988 265.10394576]
 [108.7503024  167.62652965 116.93039265]
 [ 80.1755477  159.45830807 120.11628377]]
